In [7]:
import pandas as pd
import numpy as np

from driver import Driver

In [8]:
df = pd.read_csv('data/raw/soccer/matches.csv').assign(ts = lambda _d: (pd.to_datetime(_d['date']).astype(int) / 10**9).astype(int))

In [5]:
len(df['date'].unique())

1694

In [12]:
_, bin_edges = np.histogram(sorted(df['ts'].unique()), 100)

In [16]:
bin_edges[-1]

1464134400.0

In [17]:
df['ts'].max()

1464134400